In [ ]:
import binascii
import gzip
import json
import os
import sys
import re

from io import BytesIO

import pandas as pd
import json
import altair as alt

from glob import glob

from Bio import SeqIO
from IPython.display import HTML
from onecodex.notebooks.report import set_style, title
from pathlib import Path

In [ ]:
RESULTS_DIR = Path(os.environ.get("RESULTS_DIR", os.getcwd()))

In [ ]:
# parameters
MIN_DEPTH = int(os.environ.get("MIN_DEPTH", 50))
# no longer needed?
SEQUENCING_PLATFORM = os.getenv("SEQUENCING_PLATFORM", "Oxford Nanopore")

SAMPLE_PATH = glob(str(RESULTS_DIR / "*.fastq.gz"))[0]

# outputs of bioinformatics pipeline (default paths)
VARIANTS_VCF_PATH = RESULTS_DIR / "variants.vcf"
NEXTCLADE_JSON = RESULTS_DIR / "nextclade.json"
NEXTCLADE_TSV_PATH = RESULTS_DIR / "nextclade.tsv"
PANGOLIN_CSV_PATH = RESULTS_DIR / "pangolin.csv"
CONSENSUS_PATH = RESULTS_DIR / "consensus.fa"
SNPS_DEPTH_PATH = RESULTS_DIR / "snps.depth"


# databases
REFERENCE_PATH = os.environ.get("FASTA_REFERENCE", "/share/nCoV-2019.reference.fasta")

In [ ]:
# yes this is slow but it doesn't require an API call
# (but it would be nice to not need the fastq as this point maybe. maybe this is in the bam file?)
total_reads = 0
with gzip.open(SAMPLE_PATH, "rt") as handle:
    for line in handle:
        total_reads += 1

In [ ]:
# load reference genome
reference = list(SeqIO.parse(CONSENSUS_PATH, "fasta"))
reference_length = len(reference[0])

In [ ]:
warning_messages = []

In [ ]:
### Before proceeding, do QC on the consensus sequence.
for record in SeqIO.parse(CONSENSUS_PATH, "fasta"):
    if record.seq.count("N") > 20000:
        warning_messages.append(
            "The consensus sequence has too many ambiguous bases: "
            + str(record.seq.count("N"))
            + f" N's against the {reference_length} base reference sequence."
        )
    runs = re.split(
        r"[^ATGC]", str(record.seq)
    )  # Split contig into unambiguous stretches
    max_len = len(max(runs, key=len))  # Length of longest unambiguous stretch
    if max_len < 10000:
        warning_messages.append(
            "The consensus sequence is too incomplete for GISAID submission: the longest stretch of unambiguous bases is only "
            + str(max_len)
            + " bases (must be over 10,000)."
        )

In [ ]:
# TODO: generate before
with open(RESULTS_DIR / "total_mapped_reads.txt") as handle:
    total_mapped_reads = int(handle.read())

In [ ]:
depth_table = []

with open(SNPS_DEPTH_PATH) as handle:
    for line in handle:
        row = line.strip().split("\t")
        if len(row) == 1:
            continue
        depth_table.append(
            {"reference": row[0], "position": int(row[1]), "depth": int(row[2])}
        )
depth_table = pd.DataFrame(depth_table, columns=["reference", "position", "depth"])

In [ ]:
# Calculate genome coverage (what percent of bases are coveraged at X coverage)
# Use a fixed reference length that we use for `samtools depth` above

covered_sites = set()
covered_sites_mindepth = set()

for _, row in depth_table.iterrows():
    row = row.to_dict()
    if row["depth"] >= 1:
        covered_sites.add(row["position"])
    if row["depth"] >= MIN_DEPTH:
        covered_sites_mindepth.add(row["position"])

cov = len(covered_sites) / reference_length
if cov <= 0.9:
    warning_messages.append(
        "The consensus sequence is too incomplete for GISAID submission (reads must span >90% of the reference)."
    )
cov_mindepth = len(covered_sites_mindepth) / reference_length

In [ ]:
# get mean over windows because altair can't handle > 5k points ...
binned_depths = []
window_width = reference_length // 4500

for i in range(1, reference_length, window_width):
    window = depth_table.loc[
        (depth_table["position"] > i) & (depth_table["position"] < i + window_width)
    ]

    binned_depths.append(
        {"position": i, "depth": window["depth"].mean(),}
    )

binned_depths = pd.DataFrame(binned_depths)
# Convert position from bp to kbp, to improve how the coverage plot looks
binned_depths["position"] = binned_depths["position"]/1000
mean_depth = depth_table["depth"].mean() if not depth_table.empty else 0
median_depth = depth_table["depth"].median() if not depth_table.empty else 0

In [ ]:
# Read Nextclade and Pangolin tables

if not os.path.exists(PANGOLIN_CSV_PATH):
    warning_messages.append("No pangolin output")
    have_pangolin = False
else:
    have_pangolin = True
    pangolin_table = pd.read_csv(PANGOLIN_CSV_PATH, sep=",")

In [ ]:
# Read nextclade JSON
##### Please note that everything in the Nextclade JSON (nt positions, ranges, codon positions) is 0-indexed,
##### but SARS-CoV-2 variants (and most things) are reported as 1-indexed.

with open(NEXTCLADE_JSON) as json_file:
    nextclade_json = json.load(json_file)
    assert len(nextclade_json) == 1, f"expected exactly 1 result in: {nextclade_json}"
    nextclade_json = nextclade_json[0]

In [ ]:
if len(nextclade_json["errors"]) > 0:
    have_nextclade = False
    nextclade_lineage = None
    warning_messages.extend(nextclade_json["errors"])
    n_snps_mindepth = None
    n_snps = None
    nextclade_pm_count = None
elif len(nextclade_json['substitutions']) == 0:
    warning_messages.append("No variants detected")
    have_nextclade = True
    nextclade_lineage = None
    n_snps_mindepth = 0
    n_snps = 0
    nextclade_pm_count = 0
else:
    have_nextclade = True
    # Generate warnings if indels are detected? (ONT does not reliably detect these)
    if nextclade_json["insertions"] != []:
        warning_messages.append("Insertions are detected.")
    if nextclade_json["deletions"] != []:
        warning_messages.append("Deletions are detected.")

    # load nextclade JSON
    rows_list = []
    for subst in nextclade_json["substitutions"]:  # Each substitution is a dictionary
        dict1 = {}
        dict1["Position"] = (
            subst["pos"] + 1
        )  # JSON positions are 0-indexed; convert to 1-index
        dict1["Ref"] = subst["refNuc"]
        dict1["Alt"] = subst["queryNuc"]
        if len(subst["aaSubstitutions"]) != 0:
            for mutation in subst[
                "aaSubstitutions"
            ]:  # JSON codons are 0-indexed; convert to 1-index
                dict1["Amino acid mutation"] = (
                    mutation["refAA"] + str(mutation["codon"] + 1) + mutation["queryAA"]
                )
        else:
            dict1["Amino acid mutation"] = ""
        rows_list.append(dict1)

    df_nextclade = pd.DataFrame(rows_list)
    # in case mutations are outside of Genes
    df_nextclade['Gene'] = '-'
    
    # load variants VCF
    df_vcf = pd.read_csv(
        VARIANTS_VCF_PATH,
        comment="#",
        sep="\t",
        usecols=[1, 7],
        names=["position", "info"],
        index_col=["position"],
    )
    
    # Add in gene info
    df_orfs = pd.read_csv(
        "./annot_table.orfs.txt",
        sep="\t",
        header=None,
        usecols=[0, 1, 2],
        names=["gene", "start", "stop"],
    )
    
    # join nextclade, VCF data and ORF annotations

    for i in df_nextclade.index:
        for j in df_orfs.index:
            if (
                df_orfs.loc[j, "start"]
                <= df_nextclade.loc[i, "Position"]
                <= df_orfs.loc[j, "stop"]
            ):
                df_nextclade.loc[i, "Gene"] = df_orfs.loc[j, "gene"]

    # df_nextclade in depth info
    variant_table = df_nextclade.set_index("Position")

    # parse site depth from info column
    sr = [
        [int(n) for n in x[1].split(";")[0].split(",")]
        for x in df_vcf["info"].str.rsplit(";SR=")
    ]

    assert {len(x) for x in sr} == {4}

    df_vcf["Ref depth"] = [sum(n[:2]) for n in sr]
    df_vcf["Alt depth"] = [sum(n[2:]) for n in sr]
    df_vcf["Total depth"] = df_vcf["Ref depth"] + df_vcf["Alt depth"]

    # sum depths between multiple pools
    summed = (
        df_vcf.reset_index()[["position", "Ref depth", "Alt depth", "Total depth"]]
        .groupby("position")
        .agg(sum)
    )

    summed["Alt frequency (%)"] = (
        summed["Alt depth"] / (summed["Alt depth"] + summed["Ref depth"]) * 100
    )

    pd.options.display.float_format = "{:,.2f}".format
    variants_table = variant_table.merge(
        summed, left_index=True, right_index=True, how="left"
    )

    variants_table = variants_table[
        [
            "Ref",
            "Alt",
            "Alt depth",
            "Total depth",
            "Alt frequency (%)",
            "Gene",
            "Amino acid mutation",
        ]
    ]

    n_snps = variants_table.shape[0]
    n_snps_mindepth = sum(variants_table["Total depth"] > MIN_DEPTH)

    nextclade_pm_count = nextclade_json["qc"]["privateMutations"]["total"]
    nextclade_lineage = nextclade_json["clade"]

In [ ]:
if have_pangolin:
    pangolin_lineage = pangolin_table["lineage"].iloc[0]
    pangolin_version = pangolin_table["pangoLEARN_version"].iloc[0]
else:
    warning_messages.append("Pangolin failed")
    pangolin_lineage = "Undetected or error"
    pangolin_version = "NA"

In [ ]:
title("SARS-CoV-2 (COVID-19) Sequencing Overview")

In [ ]:
text = f"""
This report summarizes the detection of SARS-CoV-2 single-nucleotide variants (SNVs) in sample 
<strong>{os.path.basename(SAMPLE_PATH)}</strong>.

<p>A minimum depth of 50x was chosen for confident SNV detection based on <a href="https://doi.org/10.1038/s41467-020-20075-6">benchmarking</a> of SARS-CoV-2 sequencing data generated with ARTIC network amplicon protocols and ONT sequencing. This benchmarking study also concludes that ONT sequencing is unsuitable for detection of small indel varants, which we do no report here.

<p>This sample contained <strong>{total_reads:,}</strong> reads, with
<strong>{total_mapped_reads / total_reads:.1%}</strong> mapping to the 
<a href='https://www.ncbi.nlm.nih.gov/nuccore/MN908947.3/' target='_blank'>Wuhan-Hu-1 reference</a>.
Reads span <strong>{cov:.0%}</strong> of the genome, with a mean depth of <strong>{mean_depth:.0f}x</strong>, and {cov_mindepth:.0%} of the genome covered at depths >{MIN_DEPTH:}x.</p>

<p>A total of <strong>{n_snps_mindepth}</strong> variant{'s were' if n_snps_mindepth != 1 else 'was'} detected at depths >{MIN_DEPTH:}x.
This genome is classified as Pangolin lineage <strong>{pangolin_lineage}</strong> using PangoLEARN version {pangolin_version} and Nextclade lineage <strong>{nextclade_lineage}</strong> with {nextclade_pm_count} private mutation{'s' if nextclade_pm_count != 1 else ''}.</p>"""

HTML(text)

In [ ]:
# Coverage plot
reference_length_kb = reference_length // 1000

plot = (
    alt.Chart(binned_depths)
    .mark_area()
    .transform_window(rolling_mean="mean(depth)", frame=[-50, 50])
    .encode(
        x=alt.X(
            "position",
            title="Genomic Coordinate (kb)",
            scale=alt.Scale(domain=[0, reference_length_kb]),
        ),
        y=alt.Y("rolling_mean:Q", scale=alt.Scale(type="linear"), title="Depth"),
    )
    .properties(
        title=f"SARS-CoV-2",
        width=550,
        height=150,
    )
)
plot

In [ ]:
if n_snps_mindepth is None:
    display(HTML("Nextclade Error. See Warnings below."))
elif n_snps_mindepth > 0: # If there are variants
    display(variants_table[variants_table['Total depth'] > MIN_DEPTH])
    legend_text = "SARS-CoV-2 variants."

    n_extra_variants = (
        sum(variants_table["Total depth"] > MIN_DEPTH) if not variant_table.empty else 0
    )

    if n_extra_variants > 0:
        legend_text += f" An additional {n_extra_variants} variant{'s' if n_extra_variants > 1 else ''} <{MIN_DEPTH}× depth {'are' if n_extra_variants > 1 else 'is'} not shown."


    if os.environ.get("ONE_CODEX_REPORT_UUID"):
        legend_text += f""" 
             A variants TSV and consensus FASTA is available <a target="_blank" href=\"{'https://app.onecodex.com/report/' + os.environ['ONE_CODEX_REPORT_UUID'] + '/files'}\">here</a>.
            """
    display(HTML(
        '<div style="text-align: center; padding-top: 10px; font-size: 0.7em; color: #777;"><em>'
        + legend_text
        + "</em></div>"
    ))
else:
    HTML(f"No variants detected > {MIN_DEPTH}-x depth")

### Additional Resources

- Additional bioinformatics pipeline details are [available on GitHub](https://github.com/onecodex/sars-cov-2)
- [Nextstrain](https://nextstrain.org/ncov) maintains an up-to-date analysis of SARS-CoV-2 (HCoV-19).
- The [Global Initiative on Sharing All Influenza Data (GISAID)](https://www.gisaid.org/) hosts viral genomes from ongoing outbreaks. Please [contact us](mailto:hello@onecodex.com) for help submitting your data.

In [ ]:
# Add One Codex report ID to footer for reproducibility/data provenance (not yet in v0.7.2)
HTML(
    f"""
<style type='text/css'>
@page {{
    @bottom-center {{
        content: "{os.environ['ONE_CODEX_REPORT_UUID'] + ' -' if os.environ.get('ONE_CODEX_REPORT_UUID') else ''} NOT FOR DIAGNOSTIC USE" !important;
    }}
}}
</style>
"""
)

In [ ]:
# Save a JSON too, including filtered variants <50x
results = {
    "n_reads": total_reads,
    "n_mapped_reads": total_mapped_reads,
    "report_id": os.environ.get("ONE_CODEX_REPORT_UUID"), 
    "sample_id": os.environ.get("ONE_CODEX_SAMPLE_UUID"),
    "variants": variants_table.to_dict(orient='records') if n_snps else None,
    "coverage": cov,
    "coverage_over_50x": cov_mindepth,
    "mean_depth": mean_depth,
    "median_depth": median_depth,
    "nextclade_results": nextclade_json,
    "pangolin_lineage": pangolin_lineage,
    "nextclade_lineage": nextclade_lineage,
    "warnings": warning_messages,
}

with gzip.open(f"{os.path.basename(SAMPLE_PATH)}.report.json.gz", "w") as f:
    f.write(json.dumps(results).encode())

In [ ]:
if len(warning_messages) > 0:
    display(HTML("<ul>"))
    display(HTML("<h1>Warning Messages</h1>"))    
    for message in warning_messages:
        display(HTML(f"<li>{message}</li>"))
    display(HTML("</ul>"))